In [1]:
!pip install flask pyngrok
!pip install nnunetv2
!pip install dicom2nifti pydicom
!apt-get update
!apt-get install -y dcm2niix
!pip install nibabel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.3/206.3 kB 14.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [68.9 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,321 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,708 kB]
Get:13 http://archive.ubuntu.com/ubuntu 

In [1]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Define nnUNet base directory
BASE_DIR = "/content/drive/MyDrive/nnUNet_data"

# Define required nnUNet paths
os.environ["nnUNet_raw"] = os.path.join(BASE_DIR, "nnUNet_raw")
os.environ["nnUNet_preprocessed"] = os.path.join(BASE_DIR, "nnUNet_preprocessed")
os.environ["nnUNet_results"] = os.path.join(BASE_DIR, "nnUNet_results")

# Verify paths
print("✅ nnUNet Paths Set:")
print(f"nnUNet_raw: {os.environ['nnUNet_raw']}")
print(f"nnUNet_preprocessed: {os.environ['nnUNet_preprocessed']}")
print(f"nnUNet_results: {os.environ['nnUNet_results']}")

Mounted at /content/drive
✅ nnUNet Paths Set:
nnUNet_raw: /content/drive/MyDrive/nnUNet_data/nnUNet_raw
nnUNet_preprocessed: /content/drive/MyDrive/nnUNet_data/nnUNet_preprocessed
nnUNet_results: /content/drive/MyDrive/nnUNet_data/nnUNet_results


In [2]:
import os
import glob
import shutil
import subprocess
import tempfile
import zipfile
from flask import Flask, request, jsonify, send_from_directory
import pydicom
import dicom2nifti
import SimpleITK as sitk
import numpy as np
import gzip
import nibabel as nib
from datetime import datetime



# Flask app setup
app = Flask(__name__)

UPLOAD_FOLDER = 'uploads'
PROCESSED_FOLDER = 'processed'
NIFTI_FOLDER = 'nifti'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(PROCESSED_FOLDER, exist_ok=True)
os.makedirs(NIFTI_FOLDER, exist_ok=True)

def is_dicom_file(file_path):
    """Check if a file is a valid DICOM file."""
    try:
        pydicom.dcmread(file_path)
        return True
    except Exception:
        return False
def extract_and_convert_dicom_to_nifti(dicom_file_path, output_dir):
    """Convert a single DICOM file to NIfTI format."""
    try:
        print(f" Processing DICOM file: {dicom_file_path}")

        # Check if the uploaded file is a valid DICOM
        if not is_dicom_file(dicom_file_path):
            raise ValueError(f"The file {dicom_file_path} is not a valid DICOM file.")
        #print(f" Valid DICOM file detected: {dicom_file_path}")

        # Convert the DICOM file to NIfTI
        nifti_file_name = os.path.join(
            output_dir, os.path.basename(dicom_file_path).replace(".dcm", "_0000.nii.gz")
        )
        sitk_image = sitk.ReadImage(dicom_file_path)
        sitk.WriteImage(sitk_image, nifti_file_name)
        print(f" DICOM converted to NIfTI: {nifti_file_name}")

        return nifti_file_name

    except Exception as e:
        print(f" ERROR in extract_and_convert_dicom_to_nifti: {e}")
        raise

def process_files(file_paths, output_dir):
    """Run nnUNetv2_predict for multiple files."""
    try:
        #print(f" Processing files: {file_paths}")

        os.environ["nnUNet_raw"] = "/content/drive/MyDrive/nnUNet_data/nnUNet_raw"
        os.environ["nnUNet_preprocessed"] = "/content/drive/MyDrive/nnUNet_data/preprocessed"
        os.environ["nnUNet_results"] = "/content/drive/MyDrive/nnUNet_data/nnUNet_results"
        print("Current Working Directory:", os.getcwd())

        command = [
            "nnUNetv2_predict",
            "-i", NIFTI_FOLDER,
            "-o", output_dir,
            "-d", "Dataset206_brainsv2",
            "-f", "0", "1", "2", "3", "4",
            "-c", "3d_fullres",
            "--disable_tta"
        ]
        print(f"🛠️ Running: {' '.join(command)}")
        result = subprocess.run(command, check=True, capture_output=True, text=True)
        print(f" nnUNet Output:\n{result.stdout}")

        predicted_files = glob.glob(os.path.join(output_dir, "*.nii.gz"))
        print(f" Predicted files: {predicted_files}")
        return predicted_files if predicted_files else None

    except Exception as e:
        print(f" ERROR in process_files: {e}")
        return None

@app.route('/')
def home():
    return "Flask Server is Running!", 200

@app.route('/predict', methods=['POST'])
def predict():
    try:
        if 'files' not in request.files:
            return jsonify({'status': 'error', 'message': 'No files uploaded'}), 400

        files = request.files.getlist('files')
        if not files or len(files) != 1:
            return jsonify({'status': 'error', 'message': 'Please upload a single DICOM or NIfTI file'}), 400

        file = files[0]
        file_path = os.path.join(UPLOAD_FOLDER, file.filename)
        file.save(file_path)

        # Check file extension
        if file.filename.endswith(".dcm"):
            #print(f" Processing DICOM file: {file.filename}")
            nifti_file = extract_and_convert_dicom_to_nifti(file_path, NIFTI_FOLDER)

        elif file.filename.endswith(".nii") or file.filename.endswith(".nii.gz"):
            #print(f" NIfTI file detected: {file.filename}")

            if file.filename.endswith(".nii"):
                try:
                    compressed_file_path = os.path.join(NIFTI_FOLDER, file.filename + ".gz")
                    print(f"🔄 Converting .nii to .nii.gz: {file_path} → {compressed_file_path}")

                    # Load the .nii file using nibabel
                    nifti_image = nib.load(file_path)

                    # Save it as .nii.gz
                    nib.save(nifti_image, compressed_file_path)

                    # Remove the original .nii file
                    os.remove(file_path)

                    # Update file_path to the new compressed file
                    file_path = compressed_file_path
                except Exception as e:
                    print(f"🔥 ERROR converting .nii to .nii.gz: {e}")
                    return jsonify({'status': 'error', 'message': 'Failed to convert .nii to .nii.gz'}), 500

            # Debug the current file_path
            print(f"📂 Updated file path: {file_path}")

            # Ensure the filename ends with _0000 before moving
            base_name, ext = os.path.splitext(file.filename)
            if ext == ".gz":  # For .nii.gz files, split twice
                base_name, ext1 = os.path.splitext(base_name)
                ext = ext1 + ext

            if not base_name.endswith("_0000"):
                base_name += "_0000"

            corrected_file_name = base_name + ext  # Preserve the correct .nii.gz extension
            if not corrected_file_name.endswith(".nii.gz"):  # Safeguard against .nii errors
                corrected_file_name += ".gz"

            nifti_file = os.path.join(NIFTI_FOLDER, corrected_file_name)
            shutil.move(file_path, nifti_file)
            #print(f"✅ NIfTI file renamed and moved to: {nifti_file}")



        else:
            return jsonify({'status': 'error', 'message': 'Unsupported file format. Only .dcm, .nii, and .nii.gz are supported.'}), 400

        # Run predictions
        predicted_files = process_files([nifti_file], PROCESSED_FOLDER)
        if predicted_files:
            segmented_files = [
                {
                    'name': os.path.basename(f),
                    'url': f"/processed/{os.path.basename(f)}"
                }
                for f in predicted_files
            ]
            return jsonify({
                'status': 'completed',
                'message': f"Processed {len(predicted_files)} files successfully.",
                'segmented_files': segmented_files
            }), 200
        else:
            return jsonify({'status': 'error', 'message': 'Processing failed.'}), 500

    except Exception as e:
        print(f"🔥 ERROR in predict: {e}")
        return jsonify({'status': 'error', 'message': str(e)}), 500

@app.route('/processed/<filename>', methods=['GET'])
def serve_processed_file(filename):
    return send_from_directory(PROCESSED_FOLDER, filename)


@app.route('/download-all', methods=['POST'])
def download_all():
    try:
        file_names = request.json.get("file_names", [])
        if not file_names:
            return jsonify({"status": "error", "message": "No files provided"}), 400

        # Check for missing files
        missing_files = [name for name in file_names if not os.path.exists(os.path.join(PROCESSED_FOLDER, name))]
        if missing_files:
            return jsonify({
                "status": "error",
                "message": f"Missing files: {', '.join(missing_files)}"
            }), 404

        # Generate a timestamp for the ZIP file name
        timestamp = datetime.now().strftime("%H%M%S")
        zip_file_name = f"segmented_files_{timestamp}.zip"
        zip_file_path = os.path.join(PROCESSED_FOLDER, zip_file_name)

        # Create the ZIP file
        with zipfile.ZipFile(zip_file_path, 'w') as zf:
            for file_name in file_names:
                file_path = os.path.join(PROCESSED_FOLDER, file_name)
                zf.write(file_path, arcname=os.path.basename(file_path))

        # Return the download URL
        return jsonify({
            "status": "success",
            "message": "ZIP file created successfully",
            "download_url": f"{request.host_url}processed/{zip_file_name}"
        })

    except Exception as e:
        print(f"🔥 ERROR in /download-all: {e}")
        return jsonify({"status": "error", "message": str(e)}), 500

In [3]:


# Start ngrok and get the public URL
from pyngrok import ngrok
auth_token = "2plIUtXATOxsvN5a1n7L5QVixr6_w1p8apzWguD5aC6ZRinW"
ngrok.set_auth_token(auth_token)

public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

# to run flask on the same thread
app.run(host="0.0.0.0", port=5000, debug=False)


Public URL: NgrokTunnel: "https://5c21-34-87-180-203.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


 Processing DICOM file: uploads/1.dcm
 DICOM converted to NIfTI: nifti/1_0000.nii.gz
Current Working Directory: /content
🛠️ Running: nnUNetv2_predict -i nifti -o processed -d Dataset206_brainsv2 -f 0 1 2 3 4 -c 3d_fullres --disable_tta


INFO:werkzeug:127.0.0.1 - - [03/Mar/2025 08:38:14] "POST /predict HTTP/1.1" 200 -


 nnUNet Output:

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

There are 1 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 1 cases that I would like to predict

Predicting 1:
perform_everything_on_device: True
sending off prediction to background worker for resampling and export
done with 1

 Predicted files: ['processed/1.nii.gz']


INFO:werkzeug:127.0.0.1 - - [03/Mar/2025 08:38:14] "POST /download-all HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Mar/2025 08:50:18] "GET /processed/segmented_files_083814.zip HTTP/1.1" 200 -
